In [6]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [4]:
### Load the trained model, scaler pickle,onehot

model= load_model("model.h5")

with open("label_encoder_gender","rb") as file:
    label_encoder_gender = pickle.load(file)

with open("onehot_encode_geography","rb") as file:
    onehot_encode_geography = pickle.load(file)

with open("scaler","rb") as file:
    scaler = pickle.load(file)


In [27]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [28]:
# One-hot encode 'Geography'
geo_encoded = onehot_encode_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encode_geography.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\salma\OneDrive\Desktop\GenAI\ANN\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:
# One-hot encode 'Geography'
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [ ]:
## Encode gender variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [ ]:
## concatination one hot encoded 
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
# Scaling 
scaled_input_df = scaler.transform(input_df)
scaled_input_df


array([[-0.56651517,  0.93698147,  0.09066692, -0.69181196, -0.27531531,
         0.83270207,  0.63444822,  0.96173168, -0.8850076 ,  1.00300451,
        -0.58119931, -0.57581067]])

In [40]:
# Run model

prediction = model.predict(scaled_input_df)
prediction

1/1 [==============================] - 0s 23ms/step


array([[0.06348914]], dtype=float32)

In [41]:
prediction_probability = prediction[0][0]
prediction_probability

0.06348914

In [42]:
if prediction_probability > 0.5:
    print('Likely to churn')
else:
    print('Not Likely to churn')

Not Likely to churn
